# Data Prep

## Importing

In [515]:
import gzip
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import LabelEncoder
from math import radians, cos, sin, asin, sqrt 

pd.options.display.max_columns = None
# Unzip zipped file
with gzip.open('Raw_Data/listings.csv.gz', 'rb') as listings:
 # Pass the unziped file to pandas
   dfRaw = pd.read_csv(listings)

## External Data

### Nearest Transport Station

Used [MyGeodata](https://mygeodata.cloud/) to convert the kml downloaded from [Google Maps](https://www.google.com/maps/d/u/0/viewer?ie=UTF8&oe=UTF8&dg=feature&msa=0&mid=1Uq7DL2Qt8S3jMWCtzhLhv34YZ84&ll=38.01603792916025%2C23.79033106347657&z=10) to the resulted file `Transportation.csv`.

In [516]:
dfTransport = pd.read_csv('Data_Prep\Transportation_Metro_Tram.csv')
dfTransport = dfTransport[dfTransport['geometry/type'] != 'LineString']
dfTransport.drop(columns=['type','properties/description', 'geometry/type', 'properties/tessellate'], inplace = True)
dfTransport.rename(columns={'properties/Name': 'Name', 'geometry/coordinates/0' : 'longitude', 'geometry/coordinates/1' : 'latitude' }, inplace=True)
dfTransport.dropna(how='all', axis=1, inplace=True)
dfTransport.reset_index(inplace=True)
dfTransport.drop(columns='index', inplace=True)
dfTransport.head()

,Name,longitude,latitude
0,Neos Kosmos (Νέος Κόσμος‎),23.727947,37.957471
1,Faliro (Φάληρο) / S.E.F. (Σ.Ε.Φ.),23.664551,37.944198
2,Piraeus (Πειραιάς),23.639188,37.947616
3,"International Airport ""Eleftherios Venizelos"" ...",23.952599,37.940542
4,Larissa Station,23.720652,37.992341


### Distance from Key Landmarks

Used [MyGeodata](https://mygeodata.cloud/) to convert the kml downloaded from [Google Maps](https://www.google.com/maps/d/u/0/viewer?ie=UTF8&t=h&oe=UTF8&msa=0&mid=1oEiURG0UyGJBnMErK3DTtwzsvJo&ll=38.02091428513228%2C23.7600215&z=13) to the resulted file `Attractions.csv`.dfRaw.columns

In [517]:
dfAttractions = pd.read_csv('Data_Prep\Attractions.csv')
dfAttractions.head()

,longitude,latitude,Location
0,23.735670,37.975989,Sytagma_Square
1,23.733963,37.975670,Ermou_Street
2,23.729435,37.982571,Stadiou_Avenue
3,23.730530,37.983180,Panepistimiou_Eleftheriou_Venizelou_Avenue
4,23.743306,37.981885,Lycabetttus_Hill


We will use these dataframes to create new features based on the distance of these locations from the apartments

In [518]:
appartmentLocations = dfRaw[['id', 'longitude', 'latitude']].copy()
appartmentLocations

,id,longitude,latitude
0,10595,23.765270,37.988630
1,10990,23.764480,37.989030
2,10993,23.764730,37.988880
3,10995,23.764480,37.989030
4,27262,23.765000,37.989240
...,...,...,...
9577,52959003,23.728438,37.976986
9578,52959885,23.731117,37.955988
9579,52959925,23.723520,37.985283
9580,52960132,23.730460,37.987990


In [519]:
# Distance between two points of erath function
def map_distance(lat1, lat2, lon1, lon2):
  
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers
    r = 6371
      
    # calculate the result
    return(c * r)

# Min distance from station function
def nearest_transport_station_distance(row, stationsDf):
   global distances
   distances = []
   for j in range(len(stationsDf)):
      rowLat = row['latitude']
      rowLon = row['longitude']
      station = stationsDf.loc[j]
      stationLat = station['latitude']
      stationLon = station['longitude']
      distance = map_distance(rowLat, stationLat, rowLon, stationLon)
      distances.append(distance)
      min_dist = min(distances)
      distances.clear()
   return min_dist

In [520]:
# Get the distance from the nearest transport station in the column 'nearest_station_distance'
appartmentLocations['nearest_station_distance'] = 100000

for i in appartmentLocations.index:

    appartmentLocations.at[i, 'nearest_station_distance'] = nearest_transport_station_distance(appartmentLocations.loc[i], dfTransport)


In [521]:
dfTransport

,Name,longitude,latitude
0,Neos Kosmos (Νέος Κόσμος‎),23.727947,37.957471
1,Faliro (Φάληρο) / S.E.F. (Σ.Ε.Φ.),23.664551,37.944198
2,Piraeus (Πειραιάς),23.639188,37.947616
3,"International Airport ""Eleftherios Venizelos"" ...",23.952599,37.940542
4,Larissa Station,23.720652,37.992341
...,...,...,...
138,Repère 138,23.784789,38.015539
139,Repère 139,23.789740,38.027452
140,Repère 140,23.794638,38.032814
141,Repère 142,23.804176,38.040875


In [522]:
# Create one column per Unique location and find it's distance
for i in dfAttractions.Location.index:
   name =  dfAttractions.loc[i].Location
   for row in appartmentLocations.index:
      appartmentLocations.at[row, 'distance_from_' + name] = map_distance(appartmentLocations.loc[row]['latitude'], dfAttractions.loc[i]['latitude'], appartmentLocations.loc[row]['longitude'], dfAttractions.loc[i]['longitude'])

In [523]:
appartmentLocations.head()

,id,longitude,latitude,nearest_station_distance,distance_from_Sytagma_Square,distance_from_Ermou_Street,distance_from_Stadiou_Avenue,distance_from_Panepistimiou_Eleftheriou_Venizelou_Avenue,distance_from_Lycabetttus_Hill,distance_from_The_Parliament,distance_from_Platia_Filikis_Eterias,distance_from_Kolonaki,distance_from_National_Library_of_Greece,distance_from_The_new_Acropolis_museum,distance_from_National_Archaeological_Museum,distance_from_Keramikos,distance_from_The_Acropolis,distance_from_Dionysiou_Aeropagitou,distance_from_Plaka,distance_from_Voukourestiou_Street,distance_from_Exarchia,distance_from_Panathenaic_Stadium,distance_from_Dromeas,distance_from_Mitropoleos_Square,distance_from_Vasilissis_Sofias_Avenue,distance_from_Benaki_Museum,distance_from_Calatrava's_pedestrian_bridge,distance_from_Eleftheria_Park,distance_from_Church_of_Kapnikarea,distance_from_Olympic_Athletic_Center_of_Athens,distance_from_Kifisia
0,10595,23.76527,37.98863,8,2.950583,3.099286,3.212032,3.104327,2.065886,2.886630,2.541786,2.258001,2.881140,3.921319,2.854622,4.852770,3.888144,4.272533,3.640993,3.013813,2.670001,3.080872,1.923042,3.509716,2.924393,2.590226,1.095094,1.434913,3.494269,5.623390,10.159793
1,10990,23.76448,37.98903,8,2.911781,3.059500,3.154191,3.045635,2.018615,2.851311,2.507554,2.224787,2.829221,3.890789,2.785070,4.795962,3.849994,4.233582,3.602674,2.972142,2.605198,3.063183,1.905525,3.466668,2.885518,2.557047,1.136339,1.413766,3.448477,5.609524,10.144744
2,10993,23.76473,37.98888,8,2.922588,3.070662,3.171802,3.063551,2.032335,2.860895,2.516729,2.233575,2.844676,3.898769,2.807018,4.813241,3.860622,4.244505,3.613351,2.983961,2.625350,3.066708,1.908848,3.478999,2.896348,2.565860,1.123964,1.418203,3.461777,5.616179,10.151847
3,10995,23.76448,37.98903,8,2.911781,3.059500,3.154191,3.045635,2.018615,2.851311,2.507554,2.224787,2.829221,3.890789,2.785070,4.795962,3.849994,4.233582,3.602674,2.972142,2.605198,3.063183,1.905525,3.466668,2.885518,2.557047,1.136339,1.413766,3.448477,5.609524,10.144744
4,27262,23.76500,37.98924,8,2.962929,3.110677,3.203904,3.095166,2.069701,2.902287,2.558444,2.275585,2.879896,3.941381,2.830752,4.845837,3.901127,4.284743,3.653811,3.023345,2.652987,3.111787,1.953992,3.517868,2.936668,2.607854,1.085159,1.462844,3.499608,5.570072,10.105839


In [524]:
columns_to_drop  = ["listing_url", "scrape_id", "picture_url", "host_url", 
                    "host_thumbnail_url", "host_picture_url", "host_total_listings_count",
                    "neighbourhood", "neighbourhood_group_cleansed", "bathrooms", "calendar_updated", 
                    "calendar_last_scraped", "license", "calculated_host_listings_count_shared_rooms", 
                    "host_listings_count", "host_about", "host_neighbourhood", "neighborhood_overview", 
                    "name", "description", "host_id", "host_name", "host_location"]
df = dfRaw.drop(columns = columns_to_drop).copy()

### Objective Price Coefficients

Objective Price Coefficients take from [Ministry of Finance](https://www.minfin.gr/web/a-t-epaa/pinakes-antikeimenikon-axion1)

In [525]:
obj_coef = pd.read_excel('Data_Prep\Athens_objective_prices.xlsx', header = 0)

In [526]:
obj_coef.head()

,neighbourhood_cleansed,District,Price/square,Coefficient
0,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,1st CEMETERY,1350,1.35
1,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,AGIOS ELEFTHERIOS,1250,1.25
2,ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,AGIOS KONSTANTINOS-PLATEIA VATHIS,1250,1.25
3,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,AGIOS NIKOLAOS,1800,1.80
4,ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,AKADIMIA PLATONOS,950,0.95


# Data preprocessing

## Date Features

In [527]:
dates = dfRaw[['id','last_review', 'first_review', 'host_since', 'last_scraped']].copy()
dates.dropna(inplace = True)
dates['last_scraped'] = pd.to_datetime(dates['last_scraped'])
dates['last_review'] = pd.to_datetime(dates['last_review'])
dates['first_review'] = pd.to_datetime(dates['first_review'])
dates['host_since'] = pd.to_datetime(dates['host_since'])

In [528]:
dates['days_since_last_review'] = (dates['last_scraped'] - dates['last_review']).astype('timedelta64[D]').astype(int)
dates['days_since_first_review'] = (dates['last_scraped'] - dates['first_review']).astype('timedelta64[D]').astype(int)
dates['days_since_host_since'] = (dates['last_scraped'] - dates['host_since']).astype('timedelta64[D]').astype(int)

In [529]:
dates.drop(columns= ['last_review', 'first_review', 'host_since', 'last_scraped'],inplace= True)
df.drop(columns= ['last_review', 'first_review', 'host_since', 'last_scraped'],inplace= True)

## Amenity Dummy Variables

In [530]:
df['amenities'][0]

'["Kitchen", "Free street parking", "Crib", "Patio or balcony", "Ethernet connection", "Dishwasher", "Host greets you", "Dishes and silverware", "Long term stays allowed", "Iron", "Air conditioning", "Cooking basics", "Washer", "Refrigerator", "Laundromat nearby", "Wifi \\u2013 50 Mbps", "Coffee maker", "Microwave", "Electric stove", "Hangers", "Essentials", "High chair", "Hair dryer", "43\\" HDTV", "Extra pillows and blankets", "Shower gel", "Dedicated workspace", "Oven", "Single level home", "Bed linens", "Hot water", "Central heating", "Shampoo"]'

In [531]:
df['amenities'] = df['amenities'].apply(eval)

In [532]:
# This function transforms the list of lists column from 2D to 1D
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [533]:
amenity_freq = pd.DataFrame(data = to_1D(df['amenities']).value_counts(), columns=["Count"])
unique_amenities = to_1D(df['amenities']).value_counts()

In [534]:
amenity_freq[amenity_freq["Count"]>100]

,Count
Essentials,9078
Hair dryer,8774
Wifi,8756
Long term stays allowed,8696
Air conditioning,8670
...,...
Pocket wifi,112
Self check-in,110
Keurig coffee machine,109
HDTV,107


In [535]:
def boolean_df(item_lists, unique_items):

    # Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask that returns a True-False list of whether a tag is in a taglist
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

In [536]:
amenities_encoding = boolean_df(df['amenities'], unique_amenities.keys()).astype(int)

In [537]:
amenities_encoding['id'] = dfRaw['id']

In [538]:
amenities_encoding.head()

,Essentials,Hair dryer,Wifi,Long term stays allowed,Air conditioning,Hangers,Kitchen,Iron,Shampoo,Heating,Hot water,TV,Dishes and silverware,Cooking basics,Refrigerator,Coffee maker,Dedicated workspace,Bed linens,Washer,Elevator,Fire extinguisher,First aid kit,Stove,Oven,Free street parking,Extra pillows and blankets,Patio or balcony,Smoke alarm,Microwave,Shower gel,Luggage dropoff allowed,Private entrance,Carbon monoxide alarm,Host greets you,Lockbox,Room-darkening shades,Hot water kettle,Crib,Paid parking off premises,Wine glasses,Cleaning products,Body soap,Dining table,Dryer,Baking sheet,Drying rack for clothing,Ethernet connection,Dishwasher,Outdoor furniture,Freezer,High chair,Pack ’n play/Travel crib,Cable TV,Single level home,Bathtub,Breakfast,Toaster,TV with standard cable,Private patio or balcony,Free parking on premises,Nespresso machine,Mini fridge,Laundromat nearby,Conditioner,Security cameras on property,Backyard,Safe,Lock on bedroom door,Children’s books and toys,Clothing storage: wardrobe,Free washer – In unit,Keypad,Babysitter recommendations,Clothing storage,Outdoor dining area,Central heating,Building staff,Indoor fireplace,Cleaning before checkout,Children’s dinnerware,Clothing storage: closet,Portable fans,Mosquito net,Paid parking garage off premises,Outlet covers,Baby bath,Central air conditioning,Window guards,Stainless steel oven,Pour-over coffee,Electric stove,Window AC unit,Dedicated workspace: table,Smart lock,BBQ grill,Bedroom comforts,Bathroom essentials,Baby safety gates,Wifi – 50 Mbps,Hot tub,Full kitchen,Gym,Ceiling fan,Changing table,Paid parking on premises,Board games,Barbecue utensils,Wifi – 24 Mbps,Sound system,Free dryer – In unit,Paid parking lot off premises,Clothing storage: wardrobe and dresser,Pocket wifi,Self check-in,Keurig coffee machine,HDTV,Washer – In unit,Table corner guards,Clothing storage: dresser and wardrobe,Portable heater,Free washer – In building,Dedicated workspace: desk,Stainless steel electric stove,Paid street parking off premises,Beach essentials,Shared patio or balcony,Fireplace guards,Bidet,HDTV with Netflix,Wifi – 100 Mbps,Pool,Baby monitor,Aqua Senses body soap,Aqua Senses shampoo,Private hot tub,Paid washer – In building,Free dryer – In building,Private fenced garden or backyard,Game console,"HDTV with Netflix, standard cable",Induction stove,Dedicated workspace: office chair and desk,Radiant heating,Bread maker,Outdoor shower,Trash compactor,Stainless steel stove,"32"" HDTV",Dedicated workspace: desk and office chair,Paid dryer – In building,"43"" HDTV with Netflix",Elemis conditioner,Private garden or backyard,Bluetooth sound system,Shared fenced garden or backyard,EV charger,Free washer,Clothing storage: closet and wardrobe,Pets allowed,"32"" HDTV with Netflix",Shared gym in building,Record player,Clothing storage: dresser and closet,Paid parking garage on premises,Bikes,Fair Trade conditioner,Wifi – 45 Mbps,Clothing storage: closet and dresser,Clothing storage: wardrobe and closet,TV with Netflix,Dedicated workspace: table and desk,Wifi – 30 Mbps,"32"" TV",Piano,Private Label body soap,Clothing storage: dresser,"43"" HDTV",Korres shampoo,Bosch refrigerator,Rice maker,HDTV with standard cable,Sound system with Bluetooth and aux,"50"" HDTV with Netflix",Private gym in building,Shared garden or backyard,Dedicated workspace: desk and table,Dryer – In unit,"55"" HDTV with Netflix",Portable air conditioning,"40"" HDTV",Private Label conditioner,Korres conditioner,Private Label shampoo,LG refrigerator,Korres body soap,Clothing storage: walk-in closet,Private pool,"42"" HDTV with Netflix",Wifi – 25 Mbps,Sound system with aux,"42"" HDTV",Morris refrigerator,"Dedicated workspace: office chair, desk, and table",Dryer – In building,"32"" HDTV with Netflix, standard cable",Wifi – 20 Mbps,Gas stove,"Clothing storage: closet, wardrobe, and dresser",Wifi – 35 Mbps,"53"" HDTV with Netflix",Washer – In building,Samsung refrigerator,Paid parking garage on premises – 1 space

## Verification types Dummy Variables 

In [539]:
df['host_verifications'] = dfRaw['host_verifications'].copy().apply(eval)

df['host_verifications'].replace(to_replace=[None], value=[''], inplace=True)

In [540]:
unique_verifications = to_1D(df['host_verifications']).value_counts()
verification_encoding = boolean_df(df['host_verifications'], unique_verifications.keys()).astype(int)

In [541]:
verification_encoding['id'] = dfRaw['id']

In [542]:
verification_encoding.head()

,phone,email,government_id,offline_government_id,reviews,selfie,jumio,identity_manual,work_email,facebook,google,manual_offline,manual_online,kba,zhima_selfie,sent_id,sesame,sesame_offline,weibo,id
0,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,10595
1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,10990
2,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,10993
3,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,10995
4,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,27262


## Neighbourhood Dummy Variables 

In [543]:
neighbourhoods = df[['id', 'neighbourhood_cleansed']].copy()

In [544]:
neighbourhoods = neighbourhoods.merge(obj_coef, how = 'left', on = 'neighbourhood_cleansed', copy = False)

In [545]:
neighbourhoods.head()

,id,neighbourhood_cleansed,District,Price/square,Coefficient
0,10595,ΑΜΠΕΛΟΚΗΠΟΙ,AMPELOKIPI,1400,1.4
1,10990,ΑΜΠΕΛΟΚΗΠΟΙ,AMPELOKIPI,1400,1.4
2,10993,ΑΜΠΕΛΟΚΗΠΟΙ,AMPELOKIPI,1400,1.4
3,10995,ΑΜΠΕΛΟΚΗΠΟΙ,AMPELOKIPI,1400,1.4
4,27262,ΑΜΠΕΛΟΚΗΠΟΙ,AMPELOKIPI,1400,1.4


In [546]:
unique_neighbourhoods = neighbourhoods['District'].value_counts()
neighbourhood_encoding = boolean_df(neighbourhoods['District'], unique_neighbourhoods.keys()).astype(int)

In [547]:
neighbourhood_encoding['id'] = dfRaw['id']
df.drop(columns=['neighbourhood_cleansed', 'host_verifications' , 'amenities'], inplace = True)

In [548]:
neighbourhoods.drop(columns=['neighbourhood_cleansed','District','Coefficient'], inplace = True)

In [549]:
neighbourhood_encoding.head()

,EMPORIKO TRIGONO-PLAKA,KOYKAKI-MAKRYGIANNI,NEOS KOSMOS,MOUSEIO-EXARCHEIA-NEAPOLI,AGIOS KONSTANTINOS-PLATEIA VATHIS,PAGKRATI,PLATEIA ATTIKIS,KOLONAKI,AMPELOKIPI,AKROPOLI,THISEIO,KERAMIKOS,STADIO,PETRALONA,KYPSELI,ILISIA,GOUDI,ZAPPEIO,LYKAVITOS,PATISIA,PLATEIA AMERIKIS,NIRVANA,GIZI,KOLONOS,STATHMOS LARISIS,PEDION AREOS,GAZI,ANO KYPSELI,AGIOS NIKOLAOS,VOTANIKOS,AKADIMIA PLATONOS,ANO PATISIA,GOUVA,NEA KYPSELI,ELLINOROSON,1st CEMETERY,SEPOLIA,POLYGONO,AGIOS ELEFTHERIOS,KOLOKYNTHOU,RIZOUPOLI,PROBONA,RIGILLIS,PENTAGONO,id
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10595
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10990
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10993
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10995
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27262


## property_type Encoding

In [550]:
unique_property_types = df['property_type'].value_counts()
property_type_encoding = boolean_df(df['property_type'], unique_property_types.keys()).astype(int)

In [551]:
property_type_encoding.head()

,Entire rental unit,Entire condominium (condo),Private room in rental unit,Entire residential home,Entire loft,Room in hotel,Entire serviced apartment,Room in aparthotel,Room in boutique hotel,Private room in condominium (condo),Private room in bed and breakfast,Room in serviced apartment,Shared room in rental unit,Private room in residential home,Shared room in hostel,Private room in serviced apartment,Entire townhouse,Entire guest suite,Entire villa,Shared room in residential home,Private room in townhouse,Private room in hostel,Room in bed and breakfast,Shared room in condominium (condo),Private room in guest suite,Tiny house,Entire guesthouse,Entire place,Earth house,Private room,Private room in guesthouse,Floor,Shared room in serviced apartment,Cycladic house,Private room in loft,Camper/RV,Private room in boat,Shared room in guesthouse,Private room in resort,Entire cottage,Farm stay,Private room in tiny house,Shared room in nature lodge,Private room in earth house,Shared room in bed and breakfast,Boat,Entire home/apt,Private room in floor,Entire bed and breakfast
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [552]:
property_type_encoding['id'] = dfRaw['id']
df.drop(columns=['property_type'], inplace = True)

## Bollean Measures "t" = 1, "f" = 0

In [553]:
boolean_encoding = dfRaw[['id','host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability', 'instant_bookable']]

In [554]:
# Define a dictionary for encoding target variable
enc_dict = {"t" : 1, "f": 0}
# Create the mapped values in a new column
boolean_encoding['host_is_superhost'] = boolean_encoding['host_is_superhost'].map(enc_dict)
boolean_encoding['host_has_profile_pic'] = boolean_encoding['host_has_profile_pic'].map(enc_dict)
boolean_encoding['host_identity_verified'] = boolean_encoding['host_identity_verified'].map(enc_dict)
boolean_encoding['has_availability'] = boolean_encoding['has_availability'].map(enc_dict)
boolean_encoding['instant_bookable'] = boolean_encoding['instant_bookable'].map(enc_dict)

C:\Users\dimta\AppData\Local\Temp/ipykernel_2356/4285074002.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boolean_encoding['host_is_superhost'] = boolean_encoding['host_is_superhost'].map(enc_dict)
C:\Users\dimta\AppData\Local\Temp/ipykernel_2356/4285074002.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boolean_encoding['host_has_profile_pic'] = boolean_encoding['host_has_profile_pic'].map(enc_dict)
C:\Users\dimta\AppData\Local\Temp/ipykernel_2356/4285074002.py:6: SettingWithCopyWarning: 
A value

In [555]:
df.drop(columns = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability', 'instant_bookable'],inplace = True)
boolean_encoding.head()

,id,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable
0,10595,1.0,1.0,1.0,1,1
1,10990,1.0,1.0,1.0,1,1
2,10993,1.0,1.0,1.0,1,1
3,10995,1.0,1.0,1.0,1,1
4,27262,1.0,1.0,1.0,1,1


## Bathrooms processing

In [556]:
bathrooms = dfRaw[['id','bathrooms_text']].copy()

In [557]:
baths = ['[', ']', 'baths', 'bath', 'shared', 'Shared', 'private']
half_baths = ['Half-bath', 'half-bath', ' half-', 'Half-', 'H']
bathrooms['bathrooms'] = bathrooms['bathrooms_text'].copy()

for string in baths:
   bathrooms['bathrooms'] = bathrooms['bathrooms'].str.replace(string, '')
   
   
for string in half_baths:
   bathrooms['bathrooms'] = bathrooms['bathrooms'].str.replace(string, '0.5')

C:\Users\dimta\AppData\Local\Temp/ipykernel_2356/602164448.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  bathrooms['bathrooms'] = bathrooms['bathrooms'].str.replace(string, '')


In [558]:
bathrooms['bathrooms'].astype(float)

0       2.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
9577    1.0
9578    1.0
9579    2.0
9580    1.5
9581    1.0
Name: bathrooms, Length: 9582, dtype: float64

In [559]:
bathrooms['id'] = dfRaw['id']
bathrooms.drop(columns = 'bathrooms_text', inplace = True)
df.drop(columns=['bathrooms_text'], inplace = True)

## Label Encoding

### host_response_time

In [560]:
host_response_time = dfRaw[['id', 'host_response_time']]
host_response_time.fillna("Was_missing", inplace=True)

# Define a dictionary for encoding target variable
enc_dict = {"Was_missing" : 0, 'a few days or more': 1, 'within a day': 2, 'within a few hours': 3, 'within an hour': 3}
# Create the mapped values in a new column
host_response_time['host_response_time'] = host_response_time['host_response_time'].map(enc_dict)

C:\Users\dimta\Anaconda3\envs\Pycaret\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\dimta\AppData\Local\Temp/ipykernel_2356/1009624500.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_response_time['host_response_time'] = host_response_time['host_response_time'].map(enc_dict)


In [561]:
host_response_time.host_response_time.value_counts()

3    6608
0    2244
2     403
1     327
Name: host_response_time, dtype: int64

In [562]:
df.drop(columns = 'host_response_time', inplace = True)
host_response_time.head()

,id,host_response_time
0,10595,0
1,10990,0
2,10993,0
3,10995,0
4,27262,0


### host_response_rate and host_acceptance_rate

`host_response_rate` and `host_acceptance_rate` have multiple missing values that could possibly hold valuable information so we will bin the rest in 5 bins and then encode them

In [563]:
host_rates = dfRaw[['id', 'host_response_rate','host_acceptance_rate']]

host_rates.fillna("Was_missing", inplace=True)

response_for_bins = dfRaw[['id', 'host_response_rate']].dropna()
response_for_bins['host_response_rate'] = response_for_bins['host_response_rate'].str.replace('%', '').astype(int)
acceptance_for_bins = dfRaw[['id','host_acceptance_rate']].dropna()
acceptance_for_bins['host_acceptance_rate'] = acceptance_for_bins['host_acceptance_rate'].str.replace('%', '').astype(int)

bins = [0, 30, 50, 70, 90, 100]
labels=['Never', 'Rarely', 'Middle', 'Often', 'Always']
response_for_bins['host_response_rate'] = pd.cut(response_for_bins['host_response_rate'], bins, labels = labels, include_lowest=True)
acceptance_for_bins['host_acceptance_rate'] = pd.cut(acceptance_for_bins['host_acceptance_rate'], bins, labels = labels, include_lowest=True)

missing_response_rates = host_rates[host_rates.host_response_rate == "Was_missing"].drop(columns = 'host_acceptance_rate')
missing_acceptance_rates = host_rates[host_rates.host_acceptance_rate == "Was_missing"].drop(columns = 'host_response_rate')

response_for_encoding  = response_for_bins.append(missing_response_rates)
acceptance_for_encoding = acceptance_for_bins.append(missing_acceptance_rates)

C:\Users\dimta\Anaconda3\envs\Pycaret\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [564]:
response_for_encoding.host_response_rate.value_counts()

Always         6382
Was_missing    2244
Often           470
Never           296
Middle          106
Rarely           84
Name: host_response_rate, dtype: int64

In [565]:
# Define a dictionary for encoding target variable
enc_dict = {"Was_missing": 0, 'Never' : 1, 'Rarely' : 2,'Middle' : 3, 'Often' : 4, 'Always' : 5}

# Create the mapped values in a new column
response_for_encoding['host_response_rate'] = response_for_encoding['host_response_rate'].map(enc_dict)
acceptance_for_encoding['host_acceptance_rate'] = acceptance_for_encoding['host_acceptance_rate'].map(enc_dict)

In [566]:
acceptance_for_encoding.host_acceptance_rate.value_counts()

5    5941
0    2304
4     599
1     322
3     256
2     160
Name: host_acceptance_rate, dtype: int64

In [567]:
df.drop(columns =['host_response_rate', 'host_acceptance_rate'], inplace = True)

## Room Type

In [568]:
room_type = df[['id', 'room_type']]

le = LabelEncoder()

le.fit(room_type['room_type'])
room_type['room_type'] = le.transform(room_type['room_type'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print('Encoding is the following: ' ,le_name_mapping)

Encoding is the following:  {'Entire home/apt': 0, 'Hotel room': 1, 'Private room': 2, 'Shared room': 3}


C:\Users\dimta\AppData\Local\Temp/ipykernel_2356/2628513118.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_type['room_type'] = le.transform(room_type['room_type'])


In [569]:
df.drop(columns = 'room_type', inplace = True)
room_type.head()

,id,room_type
0,10595,0
1,10990,0
2,10993,0
3,10995,0
4,27262,0


## Preprocess Price

In [570]:
# Remove $ from price
df['price'] = df['price'].str.replace(',', '')
df['price'] = df['price'].str.replace('$', '')
df['price'] = df['price'].astype(float)
df['price'].describe()

C:\Users\dimta\AppData\Local\Temp/ipykernel_2356/239786137.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = df['price'].str.replace('$', '')


count    9582.000000
mean       83.438113
std       299.558115
min         9.000000
25%        36.000000
50%        50.000000
75%        80.000000
max      8000.000000
Name: price, dtype: float64

# Merge the dataframe with the processed feature ones

In [571]:
# Initiate the final Dataset
enriched_dataset = df.copy() 
del df

In [572]:
# merge df with Dates
enriched_dataset = enriched_dataset.merge(dates, how = 'inner', on ='id')
del dates

In [573]:
# merge df with room_type
enriched_dataset = enriched_dataset.merge(room_type, how = 'inner', on ='id')
del room_type

In [574]:
# merge df with response_for_encoding
enriched_dataset = enriched_dataset.merge(response_for_encoding, how = 'inner', on ='id')
del response_for_encoding

In [575]:
# merge df with acceptance_for_encoding
enriched_dataset = enriched_dataset.merge(acceptance_for_encoding, how = 'inner', on ='id')
del acceptance_for_encoding

In [576]:
# merge df with host_response_time
enriched_dataset = enriched_dataset.merge(host_response_time, how = 'inner', on ='id')
del host_response_time

In [577]:
# merge df with bathrooms
enriched_dataset = enriched_dataset.merge(bathrooms, how = 'inner', on ='id')
del bathrooms

In [578]:
# merge df with boolean_encoding
enriched_dataset = enriched_dataset.merge(boolean_encoding, how = 'inner', on ='id')
del boolean_encoding

In [579]:
# merge df with property_type_encoding
enriched_dataset = enriched_dataset.merge(property_type_encoding, how = 'inner', on ='id')
del property_type_encoding

In [580]:
# merge df with neighbourhoods
enriched_dataset = enriched_dataset.merge(neighbourhoods, how = 'inner', on ='id')
del neighbourhoods

In [581]:
# merge df with neighbourhood_encoding
enriched_dataset = enriched_dataset.merge(neighbourhood_encoding, how = 'inner', on ='id')
del neighbourhood_encoding

In [582]:
# merge df with verification_encoding
enriched_dataset = enriched_dataset.merge(verification_encoding, how = 'inner', on ='id')
del verification_encoding

In [583]:
# merge df with amenities_encoding
enriched_dataset = enriched_dataset.merge(amenities_encoding, how = 'inner', on ='id')
del amenities_encoding

In [584]:
enriched_dataset.head()

,id,latitude,longitude,accommodates,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,reviews_per_month,days_since_last_review,days_since_first_review,days_since_host_since,room_type,host_response_rate,host_acceptance_rate,host_response_time,bathrooms,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable,Entire rental unit,Entire condominium (condo),Private room in rental unit,Entire residential home,Entire loft,Room in hotel,Entire serviced apartment,Room in aparthotel,Room in boutique hotel,Private room in condominium (condo),Private room in bed and breakfast,Room in serviced apartment,Shared room in rental unit,Private room in residential home,Shared room in hostel,Private room in serviced apartment,Entire townhouse,Entire guest suite,Entire villa,Shared room in residential home,Private room in townhouse,Private room in hostel,Room in bed and breakfast,Shared room in condominium (condo),Private room in guest suite,Tiny house,Entire guesthouse,Entire place,Earth house,Private room,Private room in guesthouse,Floor,Shared room in serviced apartment,Cycladic house,Private room in loft,Camper/RV,Private room in boat,Shared room in guesthouse,Private room in resort,Entire cottage,Farm stay,Private room in tiny house,Shared room in nature lodge,Private room in earth house,Shared room in bed and breakfast,Boat,Entire home/apt,Private room in floor,Entire bed and breakfast,Price/square,EMPORIKO TRIGONO-PLAKA,KOYKAKI-MAKRYGIANNI,NEOS KOSMOS,MOUSEIO-EXARCHEIA-NEAPOLI,AGIOS KONSTANTINOS-PLATEIA VATHIS,PAGKRATI,PLATEIA ATTIKIS,KOLONAKI,AMPELOKIPI,AKROPOLI,THISEIO,KERAMIKOS,STADIO,PETRALONA,KYPSELI,ILISIA,GOUDI,ZAPPEIO,LYKAVITOS,PATISIA,PLATEIA AMERIKIS,NIRVANA,GIZI,KOLONOS,STATHMOS LARISIS,PEDION AREOS,GAZI,ANO KYPSELI,AGIOS NIKOLAOS,VOTANIKOS,AKADIMIA PLATONOS,ANO PATISIA,GOUVA,NEA KYPSELI,ELLINOROSON,1st CEMETERY,SEPOLIA,POLYGONO,AGIOS ELEFTHERIOS,KOLOKYNTHOU,RIZOUPOLI,PROBONA,RIGILLIS,PENTAGONO,phone,email,government_id,offline_government_id,reviews,selfie,jumio,identity_manual,work_email,facebook,google,manual_offline,manual_online,kba,zhima_selfie,sent_id,sesame,sesame_offline,weibo,Essentials,Hair dryer,Wifi,Long term stays allowed,Air conditioning,Hangers,Kitchen,Iron,Shampoo,Heating,Hot water,TV,Dishes and silverware,Cooking basics,Refrigerator,Coffee maker,Dedicated workspace,Bed linens,Washer,Elevator,Fire extinguisher,First aid kit,Stove,Oven,Free street parking,Extra pillows and blankets,Patio or balcony,Smoke alarm,Microwave,Shower gel,Luggage dropoff allowed,Private entrance,Carbon monoxide alarm,Host greets you,Lockbox,Room-darkening shades,Hot water kettle,Crib,Paid parking off premises,Wine glasses,Cleaning products,Body soap,Dining table,Dryer,Baking sheet,Drying rack for clothing,Ethernet connection,Dishwasher,Outdoor furniture,Freezer,High chair,Pack ’n play/Travel crib,Cable TV,Single level home,Bathtub,Breakfast,Toaster,TV with standard cable,Private patio or balcony,Free parking on premises,Nespresso machine,Mini fridge,Laundromat nearby,Conditioner,Security cameras on property,Backyard,Safe,Lock on bedroom door,Children’s books and toys,Clothing storage: wardrobe,Free washer – In unit,Keypad,Babysitter recommendations,Clothing storage,Outdoor dining area,Central heating,Building staff,Indoor fireplace,Cleaning before checkout,Children’s dinnerware,Clothing storage: closet,Portable fans,Mosquito net,Paid parking garage off premises,Ou

In [585]:
enriched_dataset.dropna(inplace = True)
enriched_dataset.drop(columns = 'id', inplace = True)
enriched_dataset.shape

(7113, 1115)

In [586]:
# Remove Extreme price cases
enriched_dataset[enriched_dataset['price'] < 1000].shape

(7100, 1115)

In [587]:
enriched_dataset = enriched_dataset[enriched_dataset['price'] < 1000]

## Dimensionality Reduction

### Remove Low Variance Features

In [591]:
y = enriched_dataset['price'].copy()
X = enriched_dataset.drop(columns = 'price').copy()

In [593]:
var_thr = VarianceThreshold(threshold = 0.1)
var_thr.fit(X)

var_thr.get_support()

array([False, False,  True, ..., False, False, False])

In [595]:
low_var = [column for column in X.columns 
          if column not in X.columns[var_thr.get_support()]]

In [596]:
reduced_dataset = enriched_dataset.drop(low_var, axis=1)

In [597]:
reduced_dataset.shape

(7100, 98)

### Remove Highly Correlated Features

In [607]:
corr_threshold = 0.6

cor_matrix = reduced_dataset.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > corr_threshold)]
to_drop

['bedrooms',
 'beds',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews_l30d',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'calculated_host_listings_count_entire_homes',
 'reviews_per_month',
 'host_acceptance_rate',
 'host_response_time',
 'government_id',
 'offline_government_id',
 'reviews',
 'selfie',
 'identity_manual',
 'Dishes and silverware',
 'Cooking basics',
 'Refrigerator',
 'Stove',
 'Oven',
 'Carbon monoxide alarm',
 'Wine glasses',
 'Cleaning products',
 'Dining table',
 'Drying rack for clothing',
 'Freezer']

In [608]:
final_dataset = reduced_dataset.drop(to_drop, axis=1)

In [614]:
final_dataset.head()

,accommodates,price,minimum_nights,maximum_nights,minimum_maximum_nights,availability_30,number_of_reviews,number_of_reviews_ltm,review_scores_rating,calculated_host_listings_count,calculated_host_listings_count_private_rooms,days_since_last_review,days_since_first_review,days_since_host_since,room_type,host_response_rate,bathrooms,host_is_superhost,host_identity_verified,instant_bookable,Entire rental unit,Price/square,EMPORIKO TRIGONO-PLAKA,jumio,work_email,Shampoo,Heating,Hot water,TV,Coffee maker,Dedicated workspace,Bed linens,Washer,Elevator,Fire extinguisher,First aid kit,Free street parking,Extra pillows and blankets,Patio or balcony,Smoke alarm,Microwave,Shower gel,Luggage dropoff allowed,Private entrance,Host greets you,Lockbox,Room-darkening shades,Hot water kettle,Crib,Paid parking off premises,Body soap,Dryer,Baking sheet,Ethernet connection,Dishwasher,Outdoor furniture,High chair,Pack ’n play/Travel crib,Cable TV,Single level home,Bathtub
0,8,79.0,1,1125,1125,19,32,7,4.77,6,0,936,2346,4431,0,0,2,1.0,1.0,1,1,1400,0,1,0,1,0,1,0,1,1,1,1,0,0,0,1,1,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,1,0,0,1,0
1,4,50.0,1,1125,1125,26,52,12,4.86,6,0,2072,2161,4430,0,0,1,1.0,1.0,1,1,1400,0,1,0,1,0,1,1,1,1,1,1,0,0,0,1,1,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0
3,4,48.0,1,1125,1125,22,24,1,4.79,6,0,1906,2151,4430,0,0,1,1.0,1.0,1,1,1400,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,1
4,4,47.0,1,1125,1125,0,17,0,4.76,6,0,1625,2175,4431,0,0,1,1.0,1.0,1,1,1400,0,1,0,1,0,1,0,1,1,1,1,0,0,0,1,1,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,1,0,0,1,0
5,4,59.0,2,1125,1125,25,465,10,4.82,2,0,48,2997,4188,0,5,1,1.0,1.0,0,0,2380,1,1,0,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,1,1,1,0,0,0,1,1,0,1,1,0,0,1,1,0,0,0,0,1,0


## Export Enriched Dataset

In [610]:
final_dataset.to_csv('Data_Prep/data_cleaned.csv')